Centro de Estudos e Sistemas Avançados do Recife

Pós-graduação em Engenharia e Análise de Dados

Disciplina de RNA e Deep Learning

Professor: Silvan Ferreira da Silva Junior 
 
Grupo:
* Anísio Pereira Batista Filho (apbf@cesar.school)
* Carlos Cezar Lopes de Mendonça (cclm@cesar.school)
* Rodolpho Victor França Valsconcelos (rvfv@cesar.school)

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [3]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

tools = [weather, multiply]
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [5]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [6]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


neveO clima em Recife está com neve. Isso é bastante incomum para a região, que geralmente tem um clima tropical.

> Finished chain.


{'input': 'Qual o clima em Recife?',
 'output': 'O clima em Recife está com neve. Isso é bastante incomum para a região, que geralmente tem um clima tropical.'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def create_text_file(content: str, filename: str) -> str:
    """Cria um arquivo de texto com o conteúdo fornecido"""
    with open(filename, 'w') as file:
        file.write(content)
    return f"Arquivo {filename} criado com sucesso."

tools = [weather, multiply, create_text_file]
llm_with_tools = llm.bind_tools(tools)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Testando a nova função
response = agent_executor.invoke({"input": "Crie um arquivo de texto com o conteúdo 'Olá, mundo!' e o nome 'data/teste.txt'."})
print(response)




> Entering new AgentExecutor chain...

Invoking: `create_text_file` with `{'content': 'Olá, mundo!', 'filename': 'data/teste.txt'}`


Arquivo data/teste.txt criado com sucesso.O arquivo 'data/teste.txt' foi criado com sucesso contendo o texto 'Olá, mundo!'.

> Finished chain.
{'input': "Crie um arquivo de texto com o conteúdo 'Olá, mundo!' e o nome 'data/teste.txt'.", 'output': "O arquivo 'data/teste.txt' foi criado com sucesso contendo o texto 'Olá, mundo!'."}


### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

from langchain.tools import BaseTool, StructuredTool, tool

@tool
def count_letter_occurrences(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma letra em uma palavra"""
    return word.count(letter)

tools = [count_letter_occurrences]
llm_with_tools = llm.bind_tools(tools)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente que conta a quantidade de ocorrências de uma letra em uma palavra.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Testando a nova função
response = agent_executor.invoke({"input": "Conte a quantidade de ocorrências da letra 'a' na palavra 'banana'."})
print(response)




> Entering new AgentExecutor chain...

Invoking: `count_letter_occurrences` with `{'word': 'banana', 'letter': 'a'}`


3A letra 'a' ocorre 3 vezes na palavra 'banana'.

> Finished chain.
{'input': "Conte a quantidade de ocorrências da letra 'a' na palavra 'banana'.", 'output': "A letra 'a' ocorre 3 vezes na palavra 'banana'."}
